In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
#from geopy.distance import geodesic as GD
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pickle

import datetime
today = datetime.date.today()

# Подгрузка результатов

In [5]:
with open('CloudRun1/results_2023-05-25_2.6_1.pickle', 'rb') as f:
    cl= pd.read_pickle(f)

In [6]:
cl # результаты отработки

# Класс объекта в юпитере

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
#from geopy.distance import geodesic as GD
import warnings
warnings.filterwarnings('ignore')


class OptRouteFinder(object):

    def __init__(self, 
    path_to_times, # путь к данным о времени в пути
    path_to_coords, # путь к данным о координатах
    path_to_incomes, # путь к данным о приростах фактических (используются для подсчета затрат в конце дня) и определения баланса
    max_cash = 1000000, # лимит в терминале
    p_per_day = 0.02/365, # % на остаток в терминале
    days_limit = 14, # максимально допустимое время, в течение которого терминал можно не обслуживать
    car_cost = 20000, # стоимость одного броневика на день 
    work_day = 12*60, # рабочий день в минутах
    service_time = 10, # среднее время простоя/обслуживания
    p_service_cost=0.0001, # ставка на обслуживание
    predict_cash_trust=0.9, # доверие к прогнозу остатков (day_predict*(2-predict_cash_trust))
    num_veh = 5, # число броневиков
    cycle_time=60, # время подбора оптимального пути броневиков на 1 день
    koef_priority_0 = 10000000, # коэф. штрафа за пропуск обязательных точек для посещения 1)переполнение 2)прошло 13 дней со дня обсуживания в функции штрафа
    koef_nes_degree=3.5, # коэф. при факторе кол-во дней со дня последнего обсуживания в функции штрафа
    koef_step_func=1, # коэф. формирования ступеней для кол-ва дней со дня последнего обсуживания в функции штрафа
    koef_costs_without_vehicle=10000): # коэф. при затратах на обсуживание и % на остаток в функции штрафа
        
        #максимально допустимая сумма денег в терминале 
        self.max_cash = max_cash
        # % на остаток в терминале
        self.p_per_day = p_per_day
        #максимально допустимое время, в течение которого терминал можно не обслуживать 
        self.days_limit = days_limit
        #стоимость одного броневика на день 
        self.car_cost = car_cost
        #рабочий день в минутах
        self.work_hours = work_day
        #среднее время простоя/обслуживания
        self.service_time = service_time
        # ставка на обслуживание
        self.p_service_cost = p_service_cost
        #затраты на броневик за минуту
        self.car_cost_per_minute = self.car_cost/self.work_hours
        #доверие к прогнозу остатков
        self.predict_cash_trust = predict_cash_trust
        #расположение входных данных
        self.path_to_incomes = path_to_incomes
        self.path_to_times = path_to_times
        self.path_to_coords = path_to_coords
        
        self.num_veh = num_veh
        self.cycle_time = cycle_time
        self.koef_priority_0 = koef_priority_0
        self.koef_nes_degree =  koef_nes_degree
        self.koef_step_func = koef_step_func
        self.koef_costs_without_vehicle = koef_costs_without_vehicle
        

    def read_data(self):
        """Чтение входных данных"""
        self.coord = pd.read_excel(f'{self.path_to_coords}/terminal_data_hackathon v4.xlsx', sheet_name='TIDS', index_col=0)
        self.incomes = pd.read_excel(f'{self.path_to_incomes}/terminal_data_hackathon v4.xlsx', sheet_name='Incomes', index_col='TID')
        self.times = pd.read_csv(f'{self.path_to_times}/times v4.csv')
        
    def prepare_times(self):    
        """Подготовка данных по временным затратам"""
    
        def approx_time_cost(x, top=14):
            """Приблизительное расстояние от/до ближайшей точки через усреднение расстояния до top ближайших точек"""
            return (x.sort_values()[:top]).mean()
    
        self.time_cost = self.times.groupby('Origin_tid').agg({'Total_Time':approx_time_cost}).rename(
            columns={'Total_Time':'approx_time_cost'}) + self.times.groupby('Destination_tid').agg({'Total_Time':approx_time_cost}).rename(
            columns={'Total_Time':'approx_time_cost'}) + self.service_time
        self.time_cost['approx_time_cost_rub'] = self.time_cost.approx_time_cost*self.car_cost_per_minute

        self.times['Total_time_plus_service'] = self.times.Total_Time+self.service_time
        self.min_penalty = np.ceil(self.times['Total_time_plus_service'].max())
        
        times_c = self.times.copy()
        times_c.loc[-1]=[0,0,0,0]
        distance_matrix_df = pd.pivot_table(times_c, values='Total_time_plus_service', index=['Origin_tid'],
                                    columns=['Destination_tid'], aggfunc=np.sum).fillna(0).apply(np.round).applymap(int)
        self.distance_matrix = np.array(distance_matrix_df)

    def prepare_report_data(self):
        """Подготовка данных для симуляции"""
        
        self.balance=pd.DataFrame(data=0, index=self.incomes.index, columns = self.incomes.columns[1:])
        self.balance.insert(loc=0, column='остаток на 31.08.2022 (входящий)',
                            value=self.incomes['остаток на 31.08.2022 (входящий)'])
        self.day_df_start = pd.DataFrame(self.time_cost.approx_time_cost_rub*(-1))
        self.day_df_start.index.name ='TID'
        
        self.all_terms_set = set(self.balance.index)
        
        self.terms_dict ={0:self.all_terms_set}
        self.bags = {0:set()}
        self.day_df_dict={}
        self.route_times={}
        self.route_vehicle={}
        self.time_vehicle = {}
        
    def opt_finder(self, d=1, cycle = None):
                                       
        def service_cost(cash):
            """Издержки на обслуживание одного терминала"""
            return max(self.p_service_cost*cash, 100)
        
        def create_data_model(distance_matrix, start_route_point_pos=0, num_veh=self.num_veh):
            """Подготовка данных под задачу оптимизации"""
            data = {}
            data['distance_matrix'] = distance_matrix
            data['num_vehicles'] = num_veh
            data['depot'] = start_route_point_pos
            return data
    
        def distance_callback(from_index, to_index):
            """Определение расстояния между 2 точками"""
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]
        
        def print_solution(data, manager, routing, solution):
            """Вывод решения"""
            print(f'DAY {d}')
            print(f'Objective: {solution.ObjectiveValue()}')
            
            dropped_nodes = 'Dropped nodes:'
            dropped_nodes_lst=[]
            for node in range(routing.Size()):
                if routing.IsStart(node) or routing.IsEnd(node):
                    continue
                if solution.Value(routing.NextVar(node)) == node:
                    #dropped_nodes += ' {}'.format(manager.IndexToNode(node))
                    dropped_nodes_lst.append(manager.IndexToNode(node)-1)
            
            max_route_distance = 0
            route_vehicle = {}
            time_vehicle = {}
            for vehicle_id in range(data['num_vehicles']):
                route_vehicle[vehicle_id] = []
                index = routing.Start(vehicle_id)
                plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
                route_distance = 0
                while not routing.IsEnd(index):
                    plan_output += ' {} -> '.format(manager.IndexToNode(index))
                    route_vehicle[vehicle_id].append(manager.IndexToNode(index)-1)
                    previous_index = index
                    index = solution.Value(routing.NextVar(index))
                    route_distance += routing.GetArcCostForVehicle(
                        previous_index, index, vehicle_id)
                plan_output += '{}\n'.format(manager.IndexToNode(index))
                plan_output += 'Distance of the route: {}min\n'.format(route_distance)
                time_vehicle[vehicle_id] = route_distance + 10
                #print(plan_output)
                max_route_distance = max(route_distance, max_route_distance)
            print('Maximum of the route distances: {}min'.format(max_route_distance))
            dropped_num = 1630-len(dropped_nodes_lst)
            print(f'Точек объезда в день - {dropped_num}')
            return max_route_distance, dropped_nodes_lst, route_vehicle, time_vehicle
            
        if cycle==None:
            cycle = self.balance.shape[1]
        data = create_data_model(self.distance_matrix)
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot'])
        
        # РЕШЕНИЕ
        while d < cycle:
            day_predict = self.incomes.iloc[:,d]
            day_df = self.day_df_start.copy()
            day_df['pred_balance'] = self.balance.iloc[:,d-1] + day_predict*(2-self.predict_cash_trust)
            day_df['priority'] = (day_df.pred_balance >= self.max_cash).apply(lambda x: 0 if x==True else -1)
                
            day_df['cost_per_service'] = - day_df.pred_balance.apply(lambda x: min(x,self.max_cash)).apply(service_cost)
            day_df['cash_rate_per_day'] = day_df.pred_balance.apply(lambda x: min(x,self.max_cash))*self.p_per_day
            day_df['all_costs'] = day_df.approx_time_cost_rub + day_df.cost_per_service + day_df.cash_rate_per_day
            day_df['costs_without_vehicle'] = day_df.cost_per_service + day_df.cash_rate_per_day
            
            x=self.days_limit-1
            while x > 0:
                day_df.loc[list(set(day_df.index)&set(self.terms_dict[max(d-x,0)])), 'nes_degree'] = x
                x-=1
            day_df['nes_degree'] = day_df['nes_degree'].fillna(self.days_limit)
            
            if d >=self.days_limit:
                not_served = list(day_df[(day_df['nes_degree'] == self.days_limit)].index)
                day_df.loc[not_served, 'priority'] = 0
            
            #day_df['penalty'] = ((day_df.priority + 1)*10000000 - min_penalty*(10000/day_df.all_costs)*day_df.nes_degree**(1.5)).apply(int)
            #day_df['penalty'] = ((day_df.priority + 1)*10000000 + min_penalty*day_df.nes_degree**(-100/day_df.costs_without_vehicle+2)).apply(int)
            day_df['penalty'] = ((day_df.priority + 1)*self.koef_priority_0 -
            (self.koef_costs_without_vehicle/day_df.costs_without_vehicle)*day_df.nes_degree.apply(lambda x: x//self.koef_step_func)**(self.koef_nes_degree)).apply(int)
                                
            self.balance.iloc[:,d] = day_predict 
            #points = day_df.priority.value_counts()[0]
            
            routing = pywrapcp.RoutingModel(manager)
            transit_callback_index = routing.RegisterTransitCallback(distance_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
            
            dimension_name = 'Time'
            routing.AddDimension(
            transit_callback_index,
            0,  # нет ожидания 
            self.work_hours-self.service_time,  # максимальное время работы броневика 
            #(в матрице расстояний (в минутах) не учитывается время на обслуживание первой точки, поэтому лимит на 10 мин меньше)
            True,  # на начало рабочего дня время не потрачено
            dimension_name)
            distance_dimension = routing.GetDimensionOrDie(dimension_name)
            distance_dimension.SetGlobalSpanCostCoefficient(150)
            
            penalty =  day_df['penalty']
            for node, p in zip(range(1, len(data['distance_matrix'])), penalty):
                routing.AddDisjunction([manager.NodeToIndex(node)], p)
            
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
            search_parameters.local_search_metaheuristic = (
                routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
            #search_parameters.time_limit.FromSeconds(1)
            search_parameters.time_limit.seconds = self.cycle_time
            
            solution = routing.SolveWithParameters(search_parameters)
            
            # Решение
            if solution:
                self.route_times[d], dropped_ponts, self.route_vehicle[d], self.time_vehicle[d] = print_solution(data, manager, routing, solution)
                print(f'{d} день - Готово')
            else:
                print('Решение не найдено!')
                self.route_times[d] = np.nan
                       
            self.balance.iloc[dropped_ponts, d] = day_df.iloc[dropped_ponts, 1]
            self.terms_dict[d] = self.all_terms_set - set(day_df.iloc[dropped_ponts].index)
            self.bags[d] = set(day_df[day_df.priority==0].index) - self.terms_dict[d]
            self.day_df_dict[d]=day_df
            if len(self.bags[d]) > 0:
                print('NOT GOOD')
                break
            d+=1
            
        return self
        
def optroutefinder_lanch(path_to_times = 'data',
    path_to_coords = 'data',
    path_to_incomes = 'data',
    max_cash = 1000000,
    p_per_day = 0.02/365,
    days_limit = 14,
    car_cost = 20000,
    work_day = 12*60,
    service_time = 10,
    p_service_cost = 0.0001,
    predict_cash_trust=0.9,
    num_veh=5,
    end_day=None,
    cycle_time=300,
    koef_priority_0 = 10000000,
    koef_nes_degree=3.7,
    koef_step_func=1,
    koef_costs_without_vehicle=10000):
    
    finder = OptRouteFinder(path_to_times=path_to_times,
        path_to_coords=path_to_coords,
        path_to_incomes=path_to_incomes,
        max_cash = max_cash,
        p_per_day = p_per_day,
        days_limit = days_limit,
        car_cost = car_cost,
        work_day = work_day,
        service_time = service_time,
        p_service_cost=p_service_cost,
        predict_cash_trust=predict_cash_trust,
        num_veh=num_veh,
        cycle_time=cycle_time,
        koef_priority_0=koef_priority_0,
        koef_nes_degree=koef_nes_degree,
        koef_step_func=koef_step_func,
        koef_costs_without_vehicle=koef_costs_without_vehicle)
    
    finder.read_data()
    finder.prepare_times()
    finder.prepare_report_data()
        
    finder.opt_finder()
    return finder 

# Отработка

In [10]:
from opt_route_1 import *

In [ ]:
res = optroutefinder_lanch(path_to_times='data',
    path_to_coords='data',
    path_to_incomes='data',
    max_cash = 1000000,
    p_per_day = 0.02/365,
    days_limit = 14,
    car_cost = 20000,
    work_day = 12*60,
    service_time = 10,
    p_service_cost=0.0001,
    predict_cash_trust=0.9,
    num_veh = 5,
    cycle_time=600,
    koef_priority_0 = 10000000,
    koef_nes_degree=2.6,
    koef_step_func=1,
    koef_costs_without_vehicle=10000)

DAY 1
Objective: 283712
Maximum of the route distances: 707min
Точек объезда в день - 254
1 день - Готово
DAY 2
Objective: 1077825
Maximum of the route distances: 708min
Точек объезда в день - 161
2 день - Готово
DAY 3
Objective: 2690298
Maximum of the route distances: 710min
Точек объезда в день - 189
3 день - Готово
DAY 4
Objective: 5265830
Maximum of the route distances: 709min
Точек объезда в день - 174
4 день - Готово
DAY 5
Objective: 8318475
Maximum of the route distances: 709min
Точек объезда в день - 175
5 день - Готово
DAY 6
Objective: 12568538
Maximum of the route distances: 710min
Точек объезда в день - 182
6 день - Готово
DAY 7
Objective: 17077030
Maximum of the route distances: 709min
Точек объезда в день - 139
7 день - Готово
DAY 8
Objective: 20977589
Maximum of the route distances: 710min
Точек объезда в день - 151
8 день - Готово
DAY 9
Objective: 24741145
Maximum of the route distances: 710min
Точек объезда в день - 147
9 день - Готово
DAY 10
Objective: 28256450
Maximum